In [1]:
#### Customer support chatbot using Mistral API and FAISS
## Define categories and keywords
import os
from dotenv import load_dotenv


In [2]:
## Initialize Mistral client
from mistralai import Mistral

from dotenv import find_dotenv


def load_environment_variables():
    find_dotenv('Machine learning_projects/.env')
    MISTRAL_API = os.getenv("MISTRAL_API")
    client = Mistral(api_key=MISTRAL_API)
    return MISTRAL_API, client


In [3]:
#creating the state class
class ChatbotState:
    def __init__(self):
        self.category = None
        self.user_query = None
        self.context = None

In [4]:
load_dotenv()

CATEGORIES = {
    "Billing": ["invoice", "payment", "charged", "refund"],
    "Technical Support": ["error", "problem", "not working", "crash", "issue", "bug"],
    "General Inquiry": ["contact", "hours", "location", "address", "help"]
} 

In [5]:
## Function to categorize user queries
def categorize_query(query):
    for category, keywords in CATEGORIES.items():
        if any(keyword in query.lower() for keyword in keywords):
            return category
    return "Unknown"
## Function to get response based on category
RESPONSES = {
    "Billing": "For billing questions, please check your account settings or contact our billing support team.",
    "Technical Support": "For technical support, please try to restart your device or describe your issue in detail and our technical support team will assist you.",
    "General Inquiry": "For general inquiries, please visit our FAQ page or contact our general support team. Our business hours are 9 AM to 5 PM, Monday to Friday.",
    "Unknown": "I'm sorry, I couldn't categorize your query. Please provide more details as I connect you with a support agent."
}


In [6]:
## Function to generate response
def generate_response(query, category):
    if category in RESPONSES:
        return RESPONSES[category]
    
    # If category is unknown, use RAG to generate a response
    relevant_chunks = mistral_client.similarity_search(query, k=3)
    context = "\n".join([chunk.page_content for chunk in relevant_chunks])
    
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    
    response = mistral_client.chat.completions.create(
        model="mistral-7b-instruct",
        messages=[
            {"role": "system", "content": "You are a helpful customer support assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.7
    )
    
    return response.choices[0].message['content']

In [8]:
#Function to run customer support
def Query(user_query):
    # Load environment variables and initialize Mistral client
    MISTRAL_API, client = load_environment_variables()
    
    # Initialize chatbot state
    state = ChatbotState()
    state.user_query = user_query
    
    # Categorize the user query
    state.category = categorize_query(state.user_query)
    
    # Generate response based on category
    RESPONSES = generate_response(state.user_query, state.category)
    

    return RESPONSES

In [10]:
%pip install -U sentence-transformers


  Using cached sentence_transformers-5.1.1-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.56.2-py3-none-any.whl.metadata (40 kB)
  Using cached torch-2.8.0-cp312-cp312-win_amd64.whl.metadata (30 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl.metadata (4.1 kB)
Using cached sentence_transformers-5.1.1-py3-none-any.whl (486 kB)
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ----

ERROR: Exception:
Traceback (most recent call last):
  File "c:\Users\user\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "c:\Users\user\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\anaconda3\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\anaconda3\Lib\http\client.py", line 479, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "c:\Users\user\anaconda3\Lib\socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^

In [ ]:
# Create embeddings and FAISS index
from langchain.embeddings import HuggingFaceEmbeddings
import faiss

from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter

mistral_client = Mistral(api_key=os.getenv("MISTRAL_API"))
documents = [
    "Your invoice is available in your account settings.",
    "You can make a payment using credit card or PayPal.",
    "If you were incorrectly charged, please contact support for a refund.",
    "For technical issues, try restarting your device.",
    "If the app crashes, please report the bug to our technical team.",
    "For general inquiries, visit our FAQ page or contact support.",
    "Our business hours are 9 AM to 5 PM, Monday to Friday."
]

# Create embeddings for the documents
embedding_model = 
document_embeddings = embedding_model.embed_documents(documents)

# Create a FAISS index
faiss_index = faiss.IndexFlatL2(768)  # 768 is the dimensionality of the MiniLM embeddings
faiss_index.add(document_embeddings)

# Create a vector store using FAISS
vector_store = FAISS(embedding_model.embed_query, faiss_index, documents)
mistral_client = Mistral(api_key=os.getenv("MISTRAL_API"))

print(Query("I have an issue with my invoice and payment."))

C:\Users\user\AppData\Local\Temp\ipykernel_26596\3534416505.py:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.